About the data:
1. id: unique id for a news article
2. title: the title of a news article
3. author: author of the newa article
4. text: the text of the article; could be incomplete
5. label: a label that marks whether the news article is real or fake:

1 : fake news

0:  real news



Importing the Depencies

In [38]:
import numpy as np #useful for creating numpy arrays
import pandas as pd # useful for creating data frames and storing data in data frames
import re #useful for searching words in regular text/para(regular expression library)
from nltk.corpus import stopwords # to reduce the usage of stopwords i.e the,a
from nltk.stem.porter import PorterStemmer #provides a root word for a paticular word
from sklearn.feature_extraction.text import TfidfVectorizer #used to convert text into feature vectors
from sklearn.model_selection import train_test_split #used to split our data into training and test data
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score # Import the accuracy_score function

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processing

In [10]:
#loading the dataset to a panda DataFrame
news_dataset = pd.read_csv('/content/train.csv.zip', on_bad_lines='skip', engine='python')

In [11]:
news_dataset.shape

(20822, 5)

In [12]:
#print the first 5 rows of the dataframe
news_dataset.head()

id                                              title              author  \
0  0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1  1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2  2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3  3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4  4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...     1  
1  Ever get the feeling your life circles the rou...     0  
2  Why the Truth Might Get You Fired October 29, ...     1  
3  Videos 15 Civilians Killed In Single US Airstr...     1  
4  Print \nAn Iranian woman has been sentenced to...     1

In [13]:
# counting the number of missing values in the dataset
news_dataset.isnull().sum()

id           0
title      565
author    1975
text        59
label       22
dtype: int64

In [14]:
#replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [15]:
#merging the author name and news title
news_dataset['content']=news_dataset['author']+' '+news_dataset['title']

In [16]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20817    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20818    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20819    Michael J. de la Merced and Rachel Abrams Macy...
20820    Alex Ansary NATO, Russia To Hold Parallel Exer...
20821              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20822, dtype: object


In [17]:
#separating the data and label
x=news_dataset.drop(columns='label', axis=1)
y= news_dataset['label']

In [18]:
print(x)
print(y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20817  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20818  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20819  20797  Macy’s Is Said to Receive Takeover Approach by...   
20820  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20821  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Steming:

steming is the process of reducing a word to its root word.

example:actor,actress,acting --> act

In [19]:
port_stem = PorterStemmer()

In [20]:
def steming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = '  '.join(stemmed_content)
  return stemmed_content

In [22]:
news_dataset['content'] = news_dataset['content'].apply(steming)


In [23]:
print(news_dataset["content"])

0        darrel  lucu  hous  dem  aid  even  see  comey...
1        daniel  j  flynn  flynn  hillari  clinton  big...
2              consortiumnew  com  truth  might  get  fire
3        jessica  purkiss  civilian  kill  singl  us  a...
4        howard  portnoy  iranian  woman  jail  fiction...
                               ...                        
20817    jerom  hudson  rapper  trump  poster  child  w...
20818    benjamin  hoffman  n  f  l  playoff  schedul  ...
20819    michael  j  de  la  merc  rachel  abram  maci ...
20820    alex  ansari  nato  russia  hold  parallel  ex...
20821                        david  swanson  keep  f  aliv
Name: content, Length: 20822, dtype: object


In [25]:
#separating the data and label
X = news_dataset[ 'content'].values
Y = news_dataset['label'].values

In [26]:
print(X)

['darrel  lucu  hous  dem  aid  even  see  comey  letter  jason  chaffetz  tweet'
 'daniel  j  flynn  flynn  hillari  clinton  big  woman  campu  breitbart'
 'consortiumnew  com  truth  might  get  fire' ...
 'michael  j  de  la  merc  rachel  abram  maci  said  receiv  takeov  approach  hudson  bay  new  york  time'
 'alex  ansari  nato  russia  hold  parallel  exercis  balkan'
 'david  swanson  keep  f  aliv']


In [27]:
print(Y)

['1' '0' '1' ... '0' '1' '1']


In [28]:
Y.shape

(20822,)

In [29]:
#converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)


In [30]:
print(X)

  (0, 267)	0.27014778053672167
  (0, 2484)	0.3676990592743697
  (0, 2960)	0.24689141014750257
  (0, 3602)	0.35994096489813754
  (0, 3794)	0.2705797813233836
  (0, 4975)	0.2333632865672138
  (0, 7008)	0.21878792690522433
  (0, 7695)	0.24789859828097843
  (0, 8633)	0.29217179802324905
  (0, 8912)	0.36364344795259246
  (0, 13478)	0.2560149172431211
  (0, 15695)	0.2848972510323515
  (1, 1497)	0.2939921887587299
  (1, 1894)	0.15524631265972175
  (1, 2224)	0.3827200188088641
  (1, 2814)	0.19096625123288233
  (1, 3570)	0.2637458518155929
  (1, 5506)	0.7143145691090199
  (1, 6819)	0.1904866118084521
  (1, 16807)	0.30071934786077886
  (2, 2944)	0.3180050294894952
  (2, 3104)	0.4609666076640373
  (2, 5392)	0.3866575731579265
  (2, 5971)	0.3474726097377811
  (2, 9623)	0.4935010343775579
  :	:
  (20819, 3645)	0.21156327884082238
  (20819, 7045)	0.21799779675029382
  (20819, 8367)	0.22323238148833707
  (20819, 8991)	0.3615944630229642
  (20819, 9521)	0.2954160983178782
  (20819, 9591)	0.17456730094

Spliting the dataset into training and test data

In [33]:
# Check for class distribution
unique_classes, class_counts = np.unique(Y, return_counts=True)
print("Class Distribution:", dict(zip(unique_classes, class_counts)))

# Filter out classes with only one sample
filtered_indices = np.where(np.isin(Y, unique_classes[class_counts >= 2]))[0]
X = X[filtered_indices]
Y = Y[filtered_indices]

# Now proceed with train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)



Class Distribution: {'': 22, ' как люди воспринимают своё положение.': 1, ' чтобы это была дорога с двусторонним движением.': 1, '0': 10387, '1': 10411}


Training the model: Logistic Regression model

In [34]:
model = LogisticRegression()

In [41]:
model.fit(X_train, Y_train)

LogisticRegression()

Evaluation

accuracy score

In [39]:
#accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [40]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy scoreof the training data:  0.9922550432276657


In [42]:
#accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [43]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9824687800192123


Making a Predictive  system

In [44]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

['1']
The news is Fake


In [45]:
print(Y_test[1])

1
